In [11]:
from os import listdir
import pandas as pd
from tqdm import tqdm

dir_path= "Rohdaten_Berlin_1992-2019"
files = listdir(dir_path)


print ("alle Zählungen: {}".format(len(files)))

alle Zählungen: 20810


In [13]:
files = [s for s in files if ".zda" in s[-4:]]#.lower() ]

print ("alle Zählungen in halbwegs ordentlichen Format: {}".format(len(files)))

alle Zählungen in hlabwegs ordentlichen Format: 17000


In [14]:
files_quer = [s for s in files if "Q" in s[0]]

print ("Querschnitteszählungen: {}".format(len(files_quer)))
print ("Knotenpunktsszählungen: {}".format(len(files)-len(files_quer)))

Querschnitteszählungen: 12457
Knotenpunktsszählungen: 4543


In [15]:
#count=False
#row=0



#{'filename':f[:-4], 'knoten':knoten, 'bez':bez, 'datum':datum, 'kurzken':kurzken,
       #              'zeit':zeit, 'rich':rich, 'zaehl':cnt_dict, 'art':art, 'art_len':art_len, 'zaehl_len':zaehl_len}
df = pd.DataFrame(columns=['filename','knoten','bez','datum','kurzken','zeit','art','art_len','rich','dict_zaehl'])

for indexf,f in tqdm(enumerate(files)): #(files[:500]):
    with open(dir_path+'\\'+f, encoding='latin-1') as fi:
        content = fi.readlines()

    content = [x.strip() for x in content]
    #print (indexf)
    

    ind={}
    for index,c in enumerate(content):
    
        
        
        #### HEADER ####
        if 'knoten='  in c: 
            knoten= c.replace('knoten=','')
            df = df.append([{'filename':f[:-4]}],  ignore_index=True)
            
            df.iloc[indexf, df.columns.get_loc('knoten')] = knoten
            
        if 'bez='  in c: 
            bez= c.replace('bez=','')
            df.iloc[indexf, df.columns.get_loc('bez')] = bez

        if 'datum='  in c: 
            datum= c.replace('datum=','')
            df.iloc[indexf, df.columns.get_loc('datum')] = datum
            
        if 'zeit='  in c: 
            zeit= c.replace('zeit=','')
            
            zeit1=int(zeit[:2])
            zeit2=int(zeit[5:7])
            df.iloc[indexf, df.columns.get_loc('zeit')] = zeit
            
            #print (zeit2-zeit1)
        if 'kurzken='  in c: 
            kurzken= c.replace('kurzken=','')
            df.iloc[indexf, df.columns.get_loc('kurzken')] = kurzken
            #print (kurzken)
        if 'art='  in c: 
            art= c.replace('art=','')
            art_len=len(art.split(','))
            df.iloc[indexf, df.columns.get_loc('art')] = art
            df.iloc[indexf, df.columns.get_loc('art_len')] = art_len

        
        #### COUNTS ####
        
        richs=[]
        if '->' in c:
            
            #print (index)
            ind[c]=index

    
    dict_zaehl={}
    for index, item in enumerate(ind.items()):
        value = item[1]
        #print (len(ind))
        #print (index+1)
        if len(ind)>index+1:
            value2 = list(ind.values())[index+1]
        else:
            value2 = len(content)

        #print (value)
        #print (value2)
        key = item[0]
        key= key.replace('; ','').replace(' ','')
        
        dict_rich={}
        for indexx, c in enumerate(content[value+1:value2]):
            #dict_rich[indexx]=c
            ## replace indexx with hour
            dict_rich[indexx+zeit1]=c

            
            #print(c)

        dict_zaehl[key]=dict_rich
    #print (indexf)

    try:
        df.iloc[indexf, df.columns.get_loc('dict_zaehl')] = str(dict_zaehl)
        df.iloc[indexf, df.columns.get_loc('rich')] = dict_zaehl.keys()
    except:
        pass


    
    
df.head()  


0it [00:00, ?it/s]C:\Users\Simon\Anaconda3\envs\geo_p\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
17000it [24:09, 11.73it/s]


,filename,knoten,bez,datum,kurzken,zeit,art,art_len,rich,dict_zaehl
0,000EU7Y0,K 534,Spandauer Damm / BAB Stadtring (A 100) (Cha...,25.10.2004,K 534/04,"0700,1900","Pkw+Lfw,Lkw,Bus,Krad,Radf",5,"(1->1, 1->2, 1->3, 1->4, 2->1, 2->3, 2->4, 3->...","{'1->1': {7: '15\t6\t0\t0\t-1', 8: '12\t3\t0\t..."
1,000SR9G0,K 1035,Hohenzollerndamm / BAB Stadtring (A 100) (W...,17.06.2009,K 1035/09,"0700,1900","Pkw+Lfw,Lkw,Bus,Krad,Radf",5,"(1->1, 1->2, 1->4, 2->1, 2->3, 2->4, 3->2, 3->...","{'1->1': {7: '3\t0\t0\t0\t-1', 8: '0\t0\t0\t0\..."
2,000SZ9G1,K 1043,Berliner Straße / Konstanzer Straße - BAB A104...,18.06.2009,K 1043/09,"0700,1900","Pkw+Lfw,Lkw,Bus,Krad,Radf",5,"(1->3, 1->4, 2->1, 2->3, 2->4, 3->1, 3->4, 4->...","{'1->3': {7: '662\t18\t8\t7\t-1', 8: '893\t22\..."
3,000YQ9FZ,K 1250,Hauptstr. / Wexstr.- BAB-A100 Stadtring (Innsb...,16.06.2009,K 1250/09,"0700,1900","Pkw+Lfw,Lkw,Bus,Krad,Radf",5,"(1->1, 1->2, 1->3, 1->4, 2->1, 2->3, 2->4, 3->...","{'1->1': {7: '19\t2\t0\t0\t0', 8: '38\t1\t2\t0..."
4,0012W98P,K 1400,Saarstr. - Friedenauer Br. / BAB Abzw. Zehlend...,10.11.2008,K 1400/08,"0700,1900","Pkw+Lfw,Lkw,Bus,Krad,Radf",5,"(1->1, 1->2, 1->3, 1->4, 2->1, 2->3, 3->1, 3->...","{'1->1': {7: '9\t0\t0\t0\t0', 8: '25\t1\t0\t0\..."


In [16]:
df.to_csv(r"csv\Rohdaten_Berlin_1992-2019_zda.csv", index=False)